In [ ]:
from google.colab import drive  # Se conecta a drive
drive.mount('/content/drive')

# Imports

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, LSTM, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow import keras
import tensorflow as tf

from sklearn.metrics import confusion_matrix
from skimage.io import imread

import pandas as pd
import numpy as np
import os

# Hiperparámetros

In [ ]:
BATCH_SIZE = 32
img_size = 224
nb_crops = 3
nb_channels = 3
seed = 2021

# Carga de datos

Primero hay que cargar en el entorno de ejecución los datos generados con la libreta `Preprocesamiento_de_imágenes.ipynb`. Para ello se han comprimido todos los datos preprocesados en 5 archivos `.zip` Con estos 5 archivos hay que realizar este proceso:


1.   Se copian los 5 archivos comprimidos con las imágenes.
2.   Se descomprimen.
3.   Se eliminan los archivos comprimidos (opcional, es para ahorrar espacio).

**Importante!!!** La variable ruta debe contener la dirección en la que se han almacenado los `.zip`.

In [ ]:
ruta = './drive/MyDrive/TFG/datos/'

In [ ]:
!cp $ruta'datasetAVA_1.zip' AVADataset1.zip
!cp $ruta'datasetAVA_2.zip' AVADataset2.zip
!cp $ruta'datasetAVA_3.zip' AVADataset3.zip
!cp $ruta'datasetAVA_4.zip' AVADataset4.zip
!cp $ruta'datasetAVA_5.zip' AVADataset5.zip

In [ ]:
!unzip -q AVADataset1.zip
!unzip -q AVADataset2.zip
!unzip -q AVADataset3.zip
!unzip -q AVADataset4.zip
!unzip -q AVADataset5.zip

In [ ]:
!rm AVADataset1.zip
!rm AVADataset2.zip
!rm AVADataset3.zip
!rm AVADataset4.zip
!rm AVADataset5.zip

Se comprueba que están las 255.353 imágenes.

In [ ]:
# Se comprueba que cada carpeta tiene 10.000 imagenes. La ultima solo tiene que tener 5353
for i in range(1,27):
  print("Iteracion ", i, ". Archivos: ", len(os.listdir('./X'+str(i))))

Iteracion  1 . Archivos:  10000
Iteracion  2 . Archivos:  10000
Iteracion  3 . Archivos:  10000
Iteracion  4 . Archivos:  10000
Iteracion  5 . Archivos:  10000
Iteracion  6 . Archivos:  10000
Iteracion  7 . Archivos:  10000
Iteracion  8 . Archivos:  10000
Iteracion  9 . Archivos:  10000
Iteracion  10 . Archivos:  10000
Iteracion  11 . Archivos:  10000
Iteracion  12 . Archivos:  10000
Iteracion  13 . Archivos:  10000
Iteracion  14 . Archivos:  10000
Iteracion  15 . Archivos:  10000
Iteracion  16 . Archivos:  10000
Iteracion  17 . Archivos:  10000
Iteracion  18 . Archivos:  10000
Iteracion  19 . Archivos:  10000
Iteracion  20 . Archivos:  10000
Iteracion  21 . Archivos:  10000
Iteracion  22 . Archivos:  10000
Iteracion  23 . Archivos:  10000
Iteracion  24 . Archivos:  10000
Iteracion  25 . Archivos:  10000
Iteracion  26 . Archivos:  5353


In [ ]:
# Se cargan el dataframe con la direccion de los archivos y la variable clase
import pandas as pd
dataset = pd.read_csv(ruta+'dataset.csv')

dataset

,Unnamed: 0,id,Class
0,0,X1/0.npy,1
1,1,X1/1.npy,0
2,2,X1/2.npy,1
3,3,X1/3.npy,1
4,4,X1/4.npy,1
...,...,...,...
255348,255348,X26/5348.npy,1
255349,255349,X26/5349.npy,1
255350,255350,X26/5350.npy,1
255351,255351,X26/5351.npy,1


Se divide el dataset en train, validation y test.

In [ ]:
from sklearn.model_selection import train_test_split

tr , test = train_test_split(dataset, test_size=0.09, shuffle=True, random_state=seed, stratify=dataset['Class'])
train , validation = train_test_split(tr, test_size=0.2, shuffle=True, random_state=seed, stratify=tr['Class'])

Se define un generador que recibirá el dataframe (dirección de las parches y la variable clase) e irá cargando los 9 parches preprocesados de las imágenes.

In [ ]:
class Generator(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size=512):
        self.x = dataset['id'].to_numpy()
        self.y = dataset['Class'].to_numpy()
        self.batch_size = batch_size
        print("Se han detectado ", len(self.x), " elementos")

    def __len__(self):
        return int(np.ceil(len(self.x) / self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return np.array([np.load('./'+file_name)
               for file_name in batch_x]), np.array(batch_y)

Se define la capa de atención. Seguirá las fórmulas propuestas por Bahdanau.

In [ ]:
class Attention(Layer):

  def __init__(self, units=1):
    super(Attention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    hidden_with_time_axis = tf.expand_dims(hidden, 1)
      
    score = tf.nn.tanh(
        self.W1(features) + self.W2(hidden_with_time_axis))

    attention_weights = tf.nn.softmax(self.V(score), axis=1)

    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)
    return context_vector, attention_weights

Se aplica la técnica de submuestreo sobre el conjunto de entrenamiento y de validación.

In [ ]:
# Se separan los registros con variable clase 0 de los de variable clase 1
train_0 = train[train.Class == 0]
train_1 = train[train.Class == 1]
# Se aleatorizan los registros de la clase mayoritaria
train_1 = train_1.sample(frac = 1, random_state=seed)

# Se cogen los X primeros registros de la clase mayoritaria
# siendo X la cantidad de registros de la clase minoritaria
# para igualar la cantidad de registros que hay de ambas clases
train_nuevo = train_1[0:len(train_0)]

# Se concatenan
train2 = pd.concat([train_0, train_nuevo])
train2 = train2.sample(frac = 1, random_state=seed).reset_index(drop=True)

In [ ]:
v_0 = validation[validation.Class == 0]
v_1 = validation[validation.Class == 1]

v_1 = v_1.sample(frac = 1, random_state=seed)
v_nuevo = v_1[0:len(v_0)]

val2 = pd.concat([v_0, v_nuevo])
val2 = val2.sample(frac = 1, random_state=seed).reset_index(drop=True)

Se definen los generadores que se le pasarán como entrada a las redes neuronales.

In [ ]:
train_gen = Generator(train2)
val_gen   = Generator(val2)
test_gen  = Generator(test)

Se han detectado  107792  elementos
Se han detectado  26948  elementos
Se han detectado  22982  elementos


# Primer modelo basado en mecanismos de atención y redes LSTM

Se define el primer modelo.

In [ ]:
# Hiperparametros
d_LSTM = 0.45
units_LSTM = 9
units_attention = 10

# Modelo
shape = (nb_crops*nb_crops, 1024)
patches_procesados = tf.keras.layers.Input(shape=shape)

# Capa bidireccional con unidades LSTM
(lstm, forward_h, forward_c, backward_h, backward_c) = Bidirectional(LSTM(units_LSTM, return_sequences=True, return_state=True, dropout=d_LSTM))(patches_procesados)

# Se concatenan las h (forward y backward)
state_h = Concatenate()([forward_h, backward_h])

# Se pasan a la capa de atencion
context_vector, _ = Attention(units_attention)(lstm, state_h)

output = Dense(1, activation="sigmoid")(context_vector)

model = keras.Model(inputs=patches_procesados, outputs=output)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 9, 1024)]    0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, 9, 18), (Non 74448       input_1[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 18)           0           bidirectional[0][1]              
                                                                 bidirectional[0][3]              
__________________________________________________________________________________________________
attention (Attention)           ((None, 18), (None,  391         bidirectional[0][0]          

Antes de pasar al entrenamiento hay que compilarlo.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Se definen los callback que se le van a aplicar:


*   Early Stopping: sirve para monitorizar el entrenamiento y pararlo cuando se detecte que no haya mejora para los datos de validación.



In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)

Se entrena al modelo.

In [ ]:
epochs = 50
 
history = model.fit_generator(epochs=epochs, generator=train_gen, validation_data=val_gen, callbacks=early_stopping)

Para evaluar al modelo se calcula el `accuracy` y el `balanced accuracy` para el conjunto de datos de prueba.

In [ ]:
real = test['Class']

pred = model.predict(test_gen)

p=[]
for x in pred:
  if x > 0.5:
    p.append(1.0)
  else:
    p.append(0.0)
y_p = np.array(p)
y_p

tn, fp, fn, tp = confusion_matrix(y_true=real, y_pred=y_p).ravel()
print("Tp: ",tp," Fp: ",fp)
print("Fn: ",fn," Tn:", tn)

sensitivity = tp / (tp+fn)
specifity = tn / (fp+tn)

print("Balanced accuracy: ", (sensitivity+specifity)/2)
print("Accuracy: ", (tp+tn)/(tp+tn+fp+fn))

Se puede guardar el modelo generado para usarlo posteriormente. **Importante!** Se puede cambiar la direccion.

In [ ]:
direccion = './drive/MyDrive/TFG/model1'
tf.keras.models.save_model(model, direccion)

# Segundo modelo basado en mecanismos de atención y redes LSTM

Se define el segundo modelo.

In [ ]:
# Hiperparametros
d_LSTM = 0.45
units_LSTM = 9
units_attention = 10

# Modelo
shape = (nb_crops*nb_crops, 1024)
patches_procesados = tf.keras.layers.Input(shape=shape)

patches = tf.split(patches_procesados, num_or_size_splits=9, axis=1)

# Se agrupan los parches por filas
row1 = tf.concat([patches[0], patches[1], patches[2]], axis=1)
row2 = tf.concat([patches[3], patches[4], patches[5]], axis=1)
row3 = tf.concat([patches[6], patches[7], patches[8]], axis=1)

# Se agrupan los parches por columnas
column1 = tf.concat([patches[0], patches[3], patches[6]], axis=1)
column2 = tf.concat([patches[1], patches[4], patches[7]], axis=1)
column3 = tf.concat([patches[2], patches[5], patches[8]], axis=1)

# Se pasa cada grupo de una capa bidireccional con unidades LSTM
(r1, fh_r1, _, bh_r1, _) = Bidirectional(LSTM(units_LSTM, return_sequences=True, return_state=True))(row1)
(r2, fh_r2, _, bh_r2, _) = Bidirectional(LSTM(units_LSTM, return_sequences=True, return_state=True))(row2)
(r3, fh_r3, _, bh_r3, _) = Bidirectional(LSTM(units_LSTM, return_sequences=True, return_state=True))(row3)
(c1, fh_c1, _, bh_c1, _) = Bidirectional(LSTM(units_LSTM, return_sequences=True, return_state=True))(column1)
(c2, fh_c2, _, bh_c2, _) = Bidirectional(LSTM(units_LSTM, return_sequences=True, return_state=True))(column2)
(c3, fh_c3, _, bh_c3, _) = Bidirectional(LSTM(units_LSTM, return_sequences=True, return_state=True))(column3)

# Se concatenan las h (forward y backward)
h_r1 = Concatenate()([fh_r1, bh_r1])
h_r2 = Concatenate()([fh_r2, bh_r2])
h_r3 = Concatenate()([fh_r3, bh_r3])
h_c1 = Concatenate()([fh_c1, bh_c1])
h_c2 = Concatenate()([fh_c2, bh_c2])
h_c3 = Concatenate()([fh_c3, bh_c3])

# Se pasan a la capa de atencion --> se obtiene el vector de contexto
cv_r1, _ = Attention(units_attention)(r1, h_r1)
cv_r2, _ = Attention(units_attention)(r2, h_r2)
cv_r3, _ = Attention(units_attention)(r3, h_r3)
cv_c1, _ = Attention(units_attention)(c1, h_c1)
cv_c2, _ = Attention(units_attention)(c2, h_c2)
cv_c3, _ = Attention(units_attention)(c3, h_c3)

# Se concatenan todos los vectores del contexto
context_vector = Concatenate()([cv_r1, cv_r2, cv_r3, cv_c1, cv_c2, cv_c3])

pred = Dense(1, activation='sigmoid')(context_vector)

model2 = Model(inputs=patches_procesados, outputs=pred)

In [ ]:
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 9, 1024)]    0                                            
__________________________________________________________________________________________________
tf.split (TFOpLambda)           [(None, 1, 1024), (N 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf.concat (TFOpLambda)          (None, 3, 1024)      0           tf.split[0][0]                   
                                                                 tf.split[0][1]                   
                                                                 tf.split[0][2]                   
____________________________________________________________________________________________

Antes de pasar al entrenamiento hay que compilarlo.

In [ ]:
model2.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Se definen los callback que se le van a aplicar:


*   Early Stopping: sirve para monitorizar el entrenamiento y pararlo cuando se detecte que no haya mejora para los datos de validación.

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)

Se entrena al modelo.

In [ ]:
epochs = 50
 
history = model2.fit_generator(epochs=epochs, generator=train_gen, validation_data=val_gen, callbacks=early_stopping)

Para evaluar al modelo se calcula el `accuracy` y el `balanced accuracy` para el conjunto de datos de prueba.

In [ ]:
real = test['Class']

pred = model2.predict(test_gen)

p=[]
for x in pred:
  if x > 0.5:
    p.append(1.0)
  else:
    p.append(0.0)
y_p = np.array(p)
y_p

tn, fp, fn, tp = confusion_matrix(y_true=real, y_pred=y_p).ravel()
print("Tp: ",tp," Fp: ",fp)
print("Fn: ",fn," Tn:", tn)

sensitivity = tp / (tp+fn)
specifity = tn / (fp+tn)

print("Balanced accuracy: ", (sensitivity+specifity)/2)
print("Accuracy: ", (tp+tn)/(tp+tn+fp+fn))

Se puede guardar el modelo generado para usarlo posteriormente. **Importante!** Se puede cambiar la direccion.

In [ ]:
direccion = './drive/MyDrive/TFG/model2'
tf.keras.models.save_model(model, direccion)

# Tercer modelo basado en mecanismos de atención y redes LSTM

Se define el tercer modelo.

In [ ]:
# Hiperparametros
d_LSTM = 0.45
units_LSTM = 9
units_attention = 10

# Modelo
shape = (nb_crops*nb_crops, 1024)
patches_procesados = tf.keras.layers.Input(shape=shape)
patches = tf.split(patches_procesados, num_or_size_splits=9, axis=1)

# Se agrupan los parches por filas
row1 = tf.concat([patches[0], patches[1], patches[2]], axis=1, name='Concat_row1')
row2 = tf.concat([patches[3], patches[4], patches[5]], axis=1, name='Concat_row2')
row3 = tf.concat([patches[6], patches[7], patches[8]], axis=1, name='Concat_row3')

# Se agrupan los parches por columnas
column1 = tf.concat([patches[0], patches[3], patches[6]], axis=1)
column2 = tf.concat([patches[1], patches[4], patches[7]], axis=1)
column3 = tf.concat([patches[2], patches[5], patches[8]], axis=1)

# Se pasa cada grupo de una capa bidireccional con unidades LSTM
(r1, fh_r1, _, bh_r1, _) = Bidirectional(LSTM(units_LSTM, return_sequences=True, return_state=True, dropout=d_LSTM))(row1)
(r2, fh_r2, _, bh_r2, _) = Bidirectional(LSTM(units_LSTM, return_sequences=True, return_state=True, dropout=d_LSTM))(row2)
(r3, fh_r3, _, bh_r3, _) = Bidirectional(LSTM(units_LSTM, return_sequences=True, return_state=True, dropout=d_LSTM))(row3)
(c1, fh_c1, _, bh_c1, _) = Bidirectional(LSTM(units_LSTM, return_sequences=True, return_state=True, dropout=d_LSTM))(column1)
(c2, fh_c2, _, bh_c2, _) = Bidirectional(LSTM(units_LSTM, return_sequences=True, return_state=True, dropout=d_LSTM))(column2)
(c3, fh_c3, _, bh_c3, _) = Bidirectional(LSTM(units_LSTM, return_sequences=True, return_state=True, dropout=d_LSTM))(column3)

# Se concatenan las h (forward y backward)
h_r1 = Concatenate()([fh_r1, bh_r1])
h_r2 = Concatenate()([fh_r2, bh_r2])
h_r3 = Concatenate()([fh_r3, bh_r3])
h_c1 = Concatenate()([fh_c1, bh_c1])
h_c2 = Concatenate()([fh_c2, bh_c2])
h_c3 = Concatenate()([fh_c3, bh_c3])

# Se pasan a la capa de atencion --> se obtiene el vector de contexto
cv_r1, _ = Attention(units_attention)(r1, h_r1)
cv_r2, _ = Attention(units_attention)(r2, h_r2)
cv_r3, _ = Attention(units_attention)(r3, h_r3)
cv_c1, _ = Attention(units_attention)(c1, h_c1)
cv_c2, _ = Attention(units_attention)(c2, h_c2)
cv_c3, _ = Attention(units_attention)(c3, h_c3)

# Se concatenan todos los vectores del contexto de las filas
rows = tf.concat([tf.expand_dims(cv_r1, axis=1), tf.expand_dims(cv_r2, axis=1), tf.expand_dims(cv_r3, axis=1)], axis=1)
# Se concatenan todos los vectores del contexto de las columnas
columns = tf.concat([tf.expand_dims(cv_c1, axis=1), tf.expand_dims(cv_c2, axis=1), tf.expand_dims(cv_c3, axis=1)], axis=1)

# Se aplica la bidireccional a las filas y a las columnas
(r, fh_r, _, bh_r, _) = Bidirectional(LSTM(units_LSTM, return_sequences = True, return_state=True, dropout=d_LSTM))(rows)
(c, fh_c, _, bh_c, _) = Bidirectional(LSTM(units_LSTM, return_sequences = True, return_state=True, dropout=d_LSTM))(columns)

h_r = Concatenate()([fh_r, bh_r])
h_c = Concatenate()([fh_c, bh_c])

# Se pasan a la capa de atencion --> se obtiene el vector de contexto
cv_r, _ = Attention(units_attention)(r, h_r)
cv_c, _ = Attention(units_attention)(c, h_c)

# Se concatenan los vectores del contexto de las filas y las columnas
context_vector = Concatenate()([cv_r, cv_c])

pred = Dense(1, activation="sigmoid")(context_vector)

model3 = Model(inputs=patches_procesados, outputs=pred)

In [ ]:
model3.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 9, 1024)]    0                                            
__________________________________________________________________________________________________
tf.split_1 (TFOpLambda)         [(None, 1, 1024), (N 0           input_3[0][0]                    
__________________________________________________________________________________________________
tf.concat_6 (TFOpLambda)        (None, 3, 1024)      0           tf.split_1[0][0]                 
                                                                 tf.split_1[0][1]                 
                                                                 tf.split_1[0][2]                 
____________________________________________________________________________________________

Antes de pasar al entrenamiento hay que compilarlo.

In [ ]:
model3.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Se definen los callback que se le van a aplicar:


*   Early Stopping: sirve para monitorizar el entrenamiento y pararlo cuando se detecte que no haya mejora para los datos de validación.

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)

Se entrena al modelo.

In [ ]:
epochs = 50
 
history = model3.fit_generator(epochs=epochs, generator=train_gen, validation_data=val_gen, callbacks=early_stopping)

Para evaluar al modelo se calcula el `accuracy` y el `balanced accuracy` para el conjunto de datos de prueba.

In [ ]:
real = test['Class']

pred = model3.predict(test_gen)

p=[]
for x in pred:
  if x > 0.5:
    p.append(1.0)
  else:
    p.append(0.0)
y_p = np.array(p)
y_p

tn, fp, fn, tp = confusion_matrix(y_true=real, y_pred=y_p).ravel()
print("Tp: ",tp," Fp: ",fp)
print("Fn: ",fn," Tn:", tn)

sensitivity = tp / (tp+fn)
specifity = tn / (fp+tn)

print("Balanced accuracy: ", (sensitivity+specifity)/2)
print("Accuracy: ", (tp+tn)/(tp+tn+fp+fn))

Se puede guardar el modelo generado para usarlo posteriormente. **Importante!** Se puede cambiar la direccion.

In [ ]:
direccion = './drive/MyDrive/TFG/model3'
tf.keras.models.save_model(model, direccion)

# Carga de modelos

Se puede cargar cualquiera de los modelos guardados para reutilizarlos con la siguiente línea de código:

In [ ]:
loaded_model = tf.keras.models.load_model('./drive/MyDrive/TFG/model3')